# Autoscaling Seldon Deployments


## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - A running Kubernetes cluster with kubectl authenticated
    - The cluster should haev `heapster` and `metric-server` running in the `kube-system` namespace
 - [seldon-core Python package](https://pypi.org/project/seldon-core/) (```pip install seldon-core```)
 - [Helm client](https://helm.sh/)

### Creating a Kubernetes Cluster

Follow the [Kubernetes documentation to create a cluster](https://kubernetes.io/docs/setup/).

Once created ensure ```kubectl``` is authenticated against the running cluster.

# Setup

In [1]:
!kubectl create namespace seldon

namespace/seldon created


In [2]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "minikube" modified.


In [3]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


# Install Helm

In [4]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

serviceaccount/tiller created
clusterrolebinding.rbac.authorization.k8s.io/tiller created
$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


In [5]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

Waiting for deployment "tiller-deploy" rollout to finish: 0 of 1 updated replicas are available...
deployment "tiller-deploy" successfully rolled out


## Start seldon-core

In [6]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd --set usage_metrics.enabled=true

NAME:   seldon-core-crd
LAST DEPLOYED: Fri Feb  8 18:18:54 2019
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/ServiceAccount
NAME                        SECRETS  AGE
seldon-spartakus-volunteer  1        0s

==> v1beta1/ClusterRole
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1beta1/ClusterRoleBinding
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1/Pod(related)
NAME                                         READY  STATUS             RESTARTS  AGE
seldon-spartakus-volunteer-5554c4d8b6-j9pms  0/1    ContainerCreating  0         0s

==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  3     4s

==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  0s

==> v1beta1/Deployment
NAME                        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-spartakus-volunteer  1        1        1           0          0s


NOTES:
NOTES: TODO



In [18]:
!helm install ../../../helm-charts/seldon-core --name seldon-core --namespace seldon  --set ambassador.enabled=true

NAME:   seldon-core
LAST DEPLOYED: Fri Feb  8 18:51:12 2019
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/Pod(related)
NAME                                                 READY  STATUS             RESTARTS  AGE
seldon-core-ambassador-847f7d4c4f-6lwsj              0/1    Pending            0         0s
seldon-core-seldon-apiserver-68d9cfb85b-9c2vq        0/1    ContainerCreating  0         0s
seldon-core-seldon-cluster-manager-8558f78868-dcwhr  0/1    ContainerCreating  0         0s
seldon-core-redis-5bcfff58dc-j2b5n                   0/1    ContainerCreating  0         0s

==> v1/ServiceAccount
NAME    SECRETS  AGE
seldon  1        0s

==> v1/Role
NAME          AGE
seldon-local  0s
ambassador    0s

==> v1/RoleBinding
NAME        AGE
ambassador  0s
seldon      0s

==> v1/Service
NAME                          TYPE       CLUSTER-IP      EXTERNAL-IP  PORT(S)                        AGE
seldon-core-ambassador        NodePort   10.98.76.178    <none>       80:30616/TCP,443:30761/TCP

In [8]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-seldon-apiserver
!kubectl rollout status deploy/seldon-core-ambassador

deployment "seldon-core-seldon-cluster-manager" successfully rolled out
Waiting for deployment "seldon-core-seldon-apiserver" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-seldon-apiserver" successfully rolled out
Waiting for deployment "seldon-core-ambassador" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-ambassador" successfully rolled out


## Create model with autoscaler

To create a model with an HorizontalPodAutoscaler there are three steps:

  1. Give the PodTemplateSpec you want to autoscale a name by adding appropriate metadata, e.g.:
     ```
     "metadata":{
			"name":"my-dep"
		    },
     ```
  1. Ensure you have a resource request for the metric you want to scale on if it is a standard metric such as cpu or memory, e.g.:
     ```
      "resources": {
                                    "requests": {
                                        "cpu": "0.5"
                                    }
                                }
     ```
  1. Add a HorizontalPodAutoscalerSpec refering to this Deployment, e.g.:
     ```
     "hpaSpecs":[
		    {
			"scaleTargetRef": {			    
			    "apiVersion": "extensions/v1beta1",
			    "kind": "Deployment",
			    "name": "my-dep"},
			"minReplicas": 1,
			"maxReplicas": 4,
			"metrics": 
			    [ {
				"type": "Resource",
				"resource": {
				    "name": "cpu",
				    "targetAverageUtilization": 10
				}
			    }]
		    }],
     ```

The full SeldonDeployment spec is shown below.

In [10]:
!pygmentize model_with_hpa.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "name": "seldon-model"
    },
    "spec": {
        "name": "test-deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpecs": [{
		    "metadata":{
			"name":"my-dep"
		    },
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier:1.0",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {
                                    "requests": {
                                        "cpu": "0.5"
                                    }
                                }
                            }
                        ],
                        "termination

In [19]:
!kubectl create -f model_with_hpa.json

seldondeployment.machinelearning.seldon.io/seldon-model created


## Create Load

In [20]:
!kubectl label nodes $(kubectl get nodes -o jsonpath='{.items[0].metadata.name}') role=locust

node/minikube labeled


In [21]:
!helm install ../../../helm-charts/seldon-core-loadtesting --name loadtest  \
    --set locust.host=http://test-deployment-seldon-model:8000 \
    --set oauth.enabled=false \
    --set oauth.key=oauth-key \
    --set oauth.secret=oauth-secret \
    --set locust.hatchRate=1 \
    --set locust.clients=1 \
    --set loadtest.sendFeedback=0 \
    --set locust.minWait=0 \
    --set locust.maxWait=0 \
    --set replicaCount=1

NAME:   loadtest
LAST DEPLOYED: Fri Feb  8 18:54:47 2019
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/ReplicationController
NAME             DESIRED  CURRENT  READY  AGE
locust-slave-1   1        1        0      1s
locust-master-1  1        1        0      1s

==> v1/Service
NAME             TYPE      CLUSTER-IP      EXTERNAL-IP  PORT(S)                                       AGE
locust-master-1  NodePort  10.104.183.217  <none>       5557:32693/TCP,5558:31867/TCP,8089:31756/TCP  1s

==> v1/Pod(related)
NAME                   READY  STATUS             RESTARTS  AGE
locust-slave-1-2hth4   0/1    ContainerCreating  0         1s
locust-master-1-86ffd  0/1    ContainerCreating  0         1s




After a few mins you should see the deployment `my-dep` scaled to 4 deployments

In [22]:
!kubectl get pods,deployments,hpa

NAME                                                      READY   STATUS    RESTARTS   AGE
pod/locust-master-1-86ffd                                 1/1     Running   0          2m19s
pod/locust-slave-1-2hth4                                  1/1     Running   0          2m19s
pod/my-dep-79cdcc5ccf-2nl8z                               2/2     Running   0          5m37s
pod/my-dep-79cdcc5ccf-4xrzt                               2/2     Running   0          52s
pod/my-dep-79cdcc5ccf-rr2g9                               2/2     Running   0          52s
pod/my-dep-79cdcc5ccf-vjf8c                               2/2     Running   0          52s
pod/seldon-core-ambassador-847f7d4c4f-6lwsj               1/1     Running   0          5m54s
pod/seldon-core-redis-5bcfff58dc-j2b5n                    1/1     Running   0          5m54s
pod/seldon-core-seldon-apiserver-68d9cfb85b-9c2vq         1/1     Running   0          5m54s
pod/seldon-core-seldon-cluster-manager-8558f78868-dcwhr   1/1     Running   0 

## Remove Load
After 5-10 mins you should see the deployments replicas decrease to 1

In [23]:
!helm delete loadtest --purge

release "loadtest" deleted


In [24]:
!kubectl get pods,deployments,hpa

NAME                                                      READY   STATUS    RESTARTS   AGE
pod/my-dep-79cdcc5ccf-2nl8z                               2/2     Running   0          17m
pod/seldon-core-ambassador-847f7d4c4f-6lwsj               1/1     Running   0          17m
pod/seldon-core-redis-5bcfff58dc-j2b5n                    1/1     Running   0          17m
pod/seldon-core-seldon-apiserver-68d9cfb85b-9c2vq         1/1     Running   0          17m
pod/seldon-core-seldon-cluster-manager-8558f78868-dcwhr   1/1     Running   0          17m

NAME                                                       READY   UP-TO-DATE   AVAILABLE   AGE
deployment.extensions/my-dep                               1/1     1            1           17m
deployment.extensions/seldon-core-ambassador               1/1     1            1           17m
deployment.extensions/seldon-core-redis                    1/1     1            1           17m
deployment.extensions/seldon-core-seldon-apiserver         1/1     1 